<a href="https://colab.research.google.com/github/nishipy/CloudFunctions/blob/master/light_weight_roberta_base_scheduler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Overview
This is kernel is almost the same as [Lightweight Roberta solution in PyTorch](https://www.kaggle.com/andretugan/lightweight-roberta-solution-in-pytorch), but instead of "roberta-base", it starts from [Maunish's pre-trained model](https://www.kaggle.com/maunish/clrp-roberta-base).

Acknowledgments: some ideas were taken from kernels by [Torch](https://www.kaggle.com/rhtsingh) and [Maunish](https://www.kaggle.com/maunish).

In addition, we use the [stratified_kfold train dataset](https://www.kaggle.com/takeshikobayashi/commonlit-train-datasetfor) training the model.

## Original notebook
- Lightweight Roberta solution
  - https://www.kaggle.com/andretugan/pre-trained-roberta-solution-in-pytorch
- pretraied with MLM
  - https://www.kaggle.com/maunish/clrp-pytorch-roberta-pretrain

# Prepare

## Checking GPU status

In [121]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Sun Jul  4 12:41:58 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P0    33W / 250W |   9391MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Download dataset from kaggle

In [122]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### kaggle.json

In [123]:
!mkdir -p /root/.kaggle/
!cp ./drive/MyDrive/kaggle/commonlit/kaggle.json ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json

### Competition dataset

In [124]:
!mkdir -p ../input/commonlitreadabilityprize/
!kaggle competitions download -c commonlitreadabilityprize -p ../input/commonlitreadabilityprize/
!cp -f ./drive/MyDrive/kaggle/commonlit/train_stratiKfold.csv.zip ../input/commonlitreadabilityprize/

test.csv: Skipping, found more recently modified local copy (use --force to force download)
train.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
sample_submission.csv: Skipping, found more recently modified local copy (use --force to force download)


In [125]:
!unzip -o ../input/commonlitreadabilityprize/train.csv.zip -d ../input/commonlitreadabilityprize/
!unzip -o ../input/commonlitreadabilityprize/train_stratiKfold.csv.zip -d ../input/commonlitreadabilityprize/

Archive:  ../input/commonlitreadabilityprize/train.csv.zip
  inflating: ../input/commonlitreadabilityprize/train.csv  
Archive:  ../input/commonlitreadabilityprize/train_stratiKfold.csv.zip
  inflating: ../input/commonlitreadabilityprize/train_stratiKfold.csv  


In [126]:
!ls ../input/commonlitreadabilityprize/

pretrained-model       train.csv	      train_stratiKfold.csv.zip
sample_submission.csv  train.csv.zip
test.csv	       train_stratiKfold.csv


### Model pre-trained with MLM 
- Notebook
  - https://www.kaggle.com/maunish/clrp-pytorch-roberta-pretrain
- Model data
  - https://www.kaggle.com/maunish/clrp-roberta-base

In [127]:
!mkdir -p ../input/commonlitreadabilityprize/pretrained-model/
!kaggle datasets download maunish/clrp-roberta-base -p ../input/commonlitreadabilityprize/pretrained-model/

clrp-roberta-base.zip: Skipping, found more recently modified local copy (use --force to force download)


In [128]:
!unzip -o ../input/commonlitreadabilityprize/pretrained-model/clrp-roberta-base.zip -d ../input/commonlitreadabilityprize/pretrained-model/

Archive:  ../input/commonlitreadabilityprize/pretrained-model/clrp-roberta-base.zip
  inflating: ../input/commonlitreadabilityprize/pretrained-model/clrp_roberta_base/config.json  
  inflating: ../input/commonlitreadabilityprize/pretrained-model/clrp_roberta_base/merges.txt  
  inflating: ../input/commonlitreadabilityprize/pretrained-model/clrp_roberta_base/pytorch_model.bin  
  inflating: ../input/commonlitreadabilityprize/pretrained-model/clrp_roberta_base/special_tokens_map.json  
  inflating: ../input/commonlitreadabilityprize/pretrained-model/clrp_roberta_base/tokenizer_config.json  
  inflating: ../input/commonlitreadabilityprize/pretrained-model/clrp_roberta_base/training_args.bin  
  inflating: ../input/commonlitreadabilityprize/pretrained-model/clrp_roberta_base/vocab.json  
  inflating: ../input/commonlitreadabilityprize/pretrained-model/clrp_roberta_base_chk/checkpoint-600/config.json  
  inflating: ../input/commonlitreadabilityprize/pretrained-model/clrp_roberta_base_chk/ch

# Install dependencies

In [129]:
!pip install transformers accelerate datasets

In [130]:
import os
import math
import random
import time

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

from transformers import AdamW
from transformers import AutoTokenizer
from transformers import AutoModel
from transformers import AutoConfig
from transformers import get_cosine_schedule_with_warmup
from transformers import get_linear_schedule_with_warmup

from sklearn.model_selection import KFold

import gc
gc.enable()

# Set constant

In [131]:
NUM_FOLDS = 5
NUM_EPOCHS = 3
BATCH_SIZE = 16
MAX_LEN = 248
#(eval_rmse, step_size)
EVAL_SCHEDULE = [(0.50, 16), (0.49, 8), (0.48, 4), (0.47, 2), (-1., 1)]
ROBERTA_PATH = "../input/commonlitreadabilityprize/pretrained-model/clrp_roberta_base/"
TOKENIZER_PATH = "../input/commonlitreadabilityprize/pretrained-model/clrp_roberta_base/"
#ROBERTA_PATH = "../input/clrp-roberta-base/clrp_roberta_base"
#TOKENIZER_PATH = "../input/clrp-roberta-base/clrp_roberta_base"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

In [132]:
EVAL_SCHEDULE[0][1]

16

# Define utility functions

In [133]:
def set_random_seed(random_seed):
    random.seed(random_seed)
    np.random.seed(random_seed)
    os.environ["PYTHONHASHSEED"] = str(random_seed)

    torch.manual_seed(random_seed)
    torch.cuda.manual_seed(random_seed)
    torch.cuda.manual_seed_all(random_seed)

    torch.backends.cudnn.deterministic = True

train_dfには、Stratified kfold済みのデータセットを利用する。

In [134]:
#Use stratified k-fold train dataset
#train_df = pd.read_csv("/kaggle/input/commonlitreadabilityprize/train.csv")
train_df = pd.read_csv("../input/commonlitreadabilityprize/train_stratiKfold.csv")

# Remove incomplete entries if any.
train_df.drop(train_df[(train_df.target == 0) & (train_df.standard_error == 0)].index,
              inplace=True)
train_df.reset_index(drop=True, inplace=True)

test_df = pd.read_csv("../input/commonlitreadabilityprize/test.csv")
submission_df = pd.read_csv("../input/commonlitreadabilityprize/sample_submission.csv")

In [135]:
#TokenizerはRoberta-baseと同じ
tokenizer = AutoTokenizer.from_pretrained(TOKENIZER_PATH)

In [136]:
train_df[train_df['kfold']!=1]

,Unnamed: 0,id,url_legal,license,excerpt,target,standard_error,kfold,bins
1,1,bf24448fb,NaN,NaN,"Anywhere there is a frontier, where there are ...",-1.866238,0.510911,3,4
2,2,7cad0f936,NaN,NaN,"A great violinist, Ole Bull by name, visited t...",-0.578482,0.471768,2,6
4,4,91e87e7dc,NaN,NaN,Hans stopped snoring and awoke at supper-time....,-0.186015,0.492731,2,7
5,5,20a9f9032,NaN,NaN,The Government of the United States has viewed...,-1.391438,0.499195,4,5
6,6,daab29b47,NaN,NaN,Forty years ago women were given no representa...,-1.291128,0.531642,2,5
...,...,...,...,...,...,...,...,...,...
2826,2827,d25b7c3aa,https://www.commonlit.org/texts/the-center-of-...,CC BY-NC-SA 2.0,"The sun is a star, just like the other million...",-0.580631,0.457745,2,6
2828,2829,3c1662f6d,NaN,NaN,"It was the northwest coast of Australia, the c...",-1.678689,0.493150,2,4
2830,2831,64b635d77,https://www.africanstorybook.org/,CC BY 4.0,"Once long ago, the birds had a meeting. They w...",0.639650,0.503652,2,9
2831,2832,d6764322c,NaN,NaN,"As an adult, I might learn new actions by taki...",1.024258,0.549119,3,10


# Dataset

In [137]:
class LitDataset(Dataset):
    def __init__(self, df, inference_only=False):
        super().__init__()

        self.df = df        
        self.inference_only = inference_only
        self.text = df.excerpt.tolist()
        #改行を消してみる。元のNotebookではここはコメントアウトされている
        #self.text = [text.replace("\n", " ") for text in self.text]
        
        if not self.inference_only:
            self.target = torch.tensor(df.target.values, dtype=torch.float32)        
    
        self.encoded = tokenizer.batch_encode_plus(
            self.text,
            padding = 'max_length',            
            max_length = MAX_LEN,
            truncation = True,
            return_attention_mask=True
        )        
 

    def __len__(self):
        return len(self.df)

    
    def __getitem__(self, index):        
        input_ids = torch.tensor(self.encoded['input_ids'][index])
        attention_mask = torch.tensor(self.encoded['attention_mask'][index])
        
        if self.inference_only:
            return (input_ids, attention_mask)            
        else:
            target = self.target[index]
            return (input_ids, attention_mask, target)

# Model
The model is inspired by the one from [Maunish](https://www.kaggle.com/maunish/clrp-roberta-svm).

In [138]:
class LitModel(nn.Module):
    def __init__(self):
        super().__init__()

        config = AutoConfig.from_pretrained(ROBERTA_PATH)
        #config.jsonに書いてある設定値を更新する
        config.update({"output_hidden_states":True, 
                       "hidden_dropout_prob": 0.0,
                       "layer_norm_eps": 1e-7})                       
        
        self.roberta = AutoModel.from_pretrained(ROBERTA_PATH, config=config)  
            
        self.attention = nn.Sequential(            
            nn.Linear(768, 512),            
            nn.Tanh(),                       
            nn.Linear(512, 1),
            nn.Softmax(dim=1)
        )        

        self.regressor = nn.Sequential(                        
            nn.Linear(768, 1)                        
        )
        

    def forward(self, input_ids, attention_mask):
        roberta_output = self.roberta(input_ids=input_ids,
                                      attention_mask=attention_mask)        

        # There are a total of 13 layers of hidden states.
        # 1 for the embedding layer, and 12 for the 12 Roberta layers.
        # We take the hidden states from the last Roberta layer.
        last_layer_hidden_states = roberta_output.hidden_states[-1]

        # The number of cells is MAX_LEN.
        # The size of the hidden state of each cell is 768 (for roberta-base).
        # In order to condense hidden states of all cells to a context vector,
        # we compute a weighted average of the hidden states of all cells.
        # We compute the weight of each cell, using the attention neural network.
        weights = self.attention(last_layer_hidden_states)
                
        # weights.shape is BATCH_SIZE x MAX_LEN x 1
        # last_layer_hidden_states.shape is BATCH_SIZE x MAX_LEN x 768        
        # Now we compute context_vector as the weighted average.
        # context_vector.shape is BATCH_SIZE x 768
        context_vector = torch.sum(weights * last_layer_hidden_states, dim=1)        
        
        # Now we reduce the context vector to the prediction score.
        return self.regressor(context_vector)

## Define eval

In [139]:
#MSEで評価
def eval_mse(model, data_loader):
    """Evaluates the mean squared error of the |model| on |data_loader|"""
    model.eval()            
    mse_sum = 0

    with torch.no_grad():
        for batch_num, (input_ids, attention_mask, target) in enumerate(data_loader):
            input_ids = input_ids.to(DEVICE)
            attention_mask = attention_mask.to(DEVICE)                        
            target = target.to(DEVICE)           
            
            pred = model(input_ids, attention_mask)                       

            mse_sum += nn.MSELoss(reduction="sum")(pred.flatten(), target).item()
                

    return mse_sum / len(data_loader.dataset)

## Define predict

In [140]:
def predict(model, data_loader):
    """Returns an np.array with predictions of the |model| on |data_loader|"""
    model.eval()

    result = np.zeros(len(data_loader.dataset))    
    index = 0
    
    with torch.no_grad():
        for batch_num, (input_ids, attention_mask) in enumerate(data_loader):
            input_ids = input_ids.to(DEVICE)
            attention_mask = attention_mask.to(DEVICE)
                        
            pred = model(input_ids, attention_mask)                        

            result[index : index + pred.shape[0]] = pred.flatten().to("cpu")
            index += pred.shape[0]

    return result

### Define Train

In [141]:
def train(model, model_path, train_loader, val_loader,
          optimizer, scheduler=None, num_epochs=NUM_EPOCHS):    
    best_val_rmse = None
    best_epoch = 0
    step = 0
    last_eval_step = 0
    #EVAL_SCHEDULE = [(0.50, 16), (0.49, 8), (0.48, 4), (0.47, 2), (-1., 1)]
    #-> EVAL_SCHEDULE[0][1] = 16
    eval_period = EVAL_SCHEDULE[0][1]    

    start = time.time()

    #Epoch数だけ繰り返す
    for epoch in range(num_epochs):                           
        val_rmse = None         

        for batch_num, (input_ids, attention_mask, target) in enumerate(train_loader):
            input_ids = input_ids.to(DEVICE)
            attention_mask = attention_mask.to(DEVICE)            
            target = target.to(DEVICE)                        

            optimizer.zero_grad()
            
            model.train()

            pred = model(input_ids, attention_mask)
                                                        
            mse = nn.MSELoss(reduction="mean")(pred.flatten(), target)
                        
            mse.backward()

            #https://stackoverflow.com/questions/60120043/optimizer-and-scheduler-for-bert-fine-tuning
            #`optimizer.step()`の直後、`scheduler.step()`をすべてのバッチで呼び出して、学習率を更新します。
            optimizer.step()
            if scheduler:
                scheduler.step()
            
            #eval_period(初期値は16）stepごとにRMSEを評価
            if step >= last_eval_step + eval_period:
                # Evaluate the model on val_loader.
                elapsed_seconds = time.time() - start
                num_steps = step - last_eval_step
                print(f"\n{num_steps} steps took {elapsed_seconds:0.3} seconds")
                last_eval_step = step
                
                val_rmse = math.sqrt(eval_mse(model, val_loader))                            

                print(f"Epoch: {epoch} batch_num: {batch_num}", 
                      f"val_rmse: {val_rmse:0.4}")

                #EVAL_SCHEDULEに定義したrmseによって
                #eval_periodを変更する
                for rmse, period in EVAL_SCHEDULE:
                    if val_rmse >= rmse:
                        eval_period = period
                        break                               
                
                #ベストスコアを記録
                if not best_val_rmse or val_rmse < best_val_rmse:                    
                    best_val_rmse = val_rmse
                    best_epoch = epoch
                    torch.save(model.state_dict(), model_path)
                    print(f"New best_val_rmse: {best_val_rmse:0.4}")
                else:       
                    print(f"Still best_val_rmse: {best_val_rmse:0.4}",
                          f"(from epoch {best_epoch})")                                    
                    
                start = time.time()

            #stepをインクリメント                                          
            step += 1
                        
    
    return best_val_rmse

## Create Optimizer

In [142]:
def create_optimizer(model):
    named_parameters = list(model.named_parameters())    
    
    roberta_parameters = named_parameters[:197]    
    attention_parameters = named_parameters[199:203]
    regressor_parameters = named_parameters[203:]
        
    attention_group = [params for (name, params) in attention_parameters]
    regressor_group = [params for (name, params) in regressor_parameters]

    parameters = []
    parameters.append({"params": attention_group})
    parameters.append({"params": regressor_group})

    for layer_num, (name, params) in enumerate(roberta_parameters):
        weight_decay = 0.0 if "bias" in name else 0.01

        lr = 2e-5

        if layer_num >= 69:        
            lr = 5e-5

        if layer_num >= 133:
            lr = 1e-4

        parameters.append({"params": params,
                           "weight_decay": weight_decay,
                           "lr": lr})

    return AdamW(parameters)

## Run

In [143]:
gc.collect()

SEED = 1000
list_val_rmse = []

for fold in range(NUM_FOLDS): 
    print(f"\nFold {fold + 1}/{NUM_FOLDS}")
    model_path = f"model_{fold + 1}.pth"
        
    set_random_seed(SEED + fold)

    #Stratified kfold train dataset用に修正
    train_dataset = LitDataset(train_df[train_df['kfold']!=fold])    
    val_dataset = LitDataset(train_df[train_df['kfold']==fold])    
    
    #https://pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE,
                              drop_last=True, shuffle=True, num_workers=2)    
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE,
                            drop_last=False, shuffle=False, num_workers=2)    
    
    #random_seedは、Foldごとに変わる
    set_random_seed(SEED + fold)    
    
    model = LitModel().to(DEVICE)
    
    optimizer = create_optimizer(model)
    #Schedulerには、get_cosine_schedule_with_warmupを使っている
    #その他の選択肢: https://huggingface.co/transformers/main_classes/optimizer_schedules.html#schedules                        
    # scheduler = get_cosine_schedule_with_warmup(
    #     optimizer,
    #     num_training_steps=NUM_EPOCHS * len(train_loader),
    #     num_warmup_steps=50)
    #num_warmup_steps: num_training_stepsの6%を設定してみる
    # -> ceil(3 * 2266 * 0.06) = 408
    # https://www.kaggle.com/c/commonlitreadabilityprize/discussion/241029
    # -> ceil(2266 * 0.06) = 136
    scheduler = get_linear_schedule_with_warmup(
        optimizer,
        num_training_steps=NUM_EPOCHS * len(train_loader),
        num_warmup_steps=25) 

    list_val_rmse.append(train(model, model_path, train_loader,
                               val_loader, optimizer, scheduler=scheduler))

    del model
    gc.collect()
    
    print("\nPerformance estimates:")
    print(list_val_rmse)
    print("Mean:", np.array(list_val_rmse).mean())

    


Fold 1/5


Some weights of the model checkpoint at ../input/commonlitreadabilityprize/pretrained-model/clrp_roberta_base/ were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.decoder.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at ../input/commonlitreadabilityprize/pretrained-model/clrp_roberta_base/ and are newly initialized: ['roberta.pooler.dense.weight', 'rober


16 steps took 7.05 seconds
Epoch: 0 batch_num: 16 val_rmse: 0.9481
New best_val_rmse: 0.9481

16 steps took 6.33 seconds
Epoch: 0 batch_num: 32 val_rmse: 0.6801
New best_val_rmse: 0.6801

16 steps took 6.33 seconds
Epoch: 0 batch_num: 48 val_rmse: 0.6885
Still best_val_rmse: 0.6801 (from epoch 0)

16 steps took 6.33 seconds
Epoch: 0 batch_num: 64 val_rmse: 0.6136
New best_val_rmse: 0.6136

16 steps took 6.33 seconds
Epoch: 0 batch_num: 80 val_rmse: 0.6261
Still best_val_rmse: 0.6136 (from epoch 0)

16 steps took 6.33 seconds
Epoch: 0 batch_num: 96 val_rmse: 0.5746
New best_val_rmse: 0.5746

16 steps took 6.33 seconds
Epoch: 0 batch_num: 112 val_rmse: 0.5257
New best_val_rmse: 0.5257

16 steps took 6.33 seconds
Epoch: 0 batch_num: 128 val_rmse: 0.5208
New best_val_rmse: 0.5208

16 steps took 6.53 seconds
Epoch: 1 batch_num: 3 val_rmse: 0.5121
New best_val_rmse: 0.5121

16 steps took 6.33 seconds
Epoch: 1 batch_num: 19 val_rmse: 0.5213
Still best_val_rmse: 0.5121 (from epoch 1)

16 step

Some weights of the model checkpoint at ../input/commonlitreadabilityprize/pretrained-model/clrp_roberta_base/ were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.decoder.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at ../input/commonlitreadabilityprize/pretrained-model/clrp_roberta_base/ and are newly initialized: ['roberta.pooler.dense.weight', 'rober


16 steps took 6.87 seconds
Epoch: 0 batch_num: 16 val_rmse: 0.9592
New best_val_rmse: 0.9592

16 steps took 6.33 seconds
Epoch: 0 batch_num: 32 val_rmse: 0.644
New best_val_rmse: 0.644

16 steps took 6.33 seconds
Epoch: 0 batch_num: 48 val_rmse: 0.619
New best_val_rmse: 0.619

16 steps took 6.33 seconds
Epoch: 0 batch_num: 64 val_rmse: 0.7022
Still best_val_rmse: 0.619 (from epoch 0)

16 steps took 6.33 seconds
Epoch: 0 batch_num: 80 val_rmse: 0.5665
New best_val_rmse: 0.5665

16 steps took 6.33 seconds
Epoch: 0 batch_num: 96 val_rmse: 0.5574
New best_val_rmse: 0.5574

16 steps took 6.33 seconds
Epoch: 0 batch_num: 112 val_rmse: 0.5401
New best_val_rmse: 0.5401

16 steps took 6.33 seconds
Epoch: 0 batch_num: 128 val_rmse: 0.5627
Still best_val_rmse: 0.5401 (from epoch 0)

16 steps took 6.52 seconds
Epoch: 1 batch_num: 3 val_rmse: 0.5155
New best_val_rmse: 0.5155

16 steps took 6.32 seconds
Epoch: 1 batch_num: 19 val_rmse: 0.5034
New best_val_rmse: 0.5034

16 steps took 6.33 seconds
Ep

...


16 steps took 6.86 seconds
Epoch: 0 batch_num: 16 val_rmse: 0.9665
New best_val_rmse: 0.9665

16 steps took 6.33 seconds
Epoch: 0 batch_num: 32 val_rmse: 0.6752
New best_val_rmse: 0.6752

16 steps took 6.32 seconds
Epoch: 0 batch_num: 48 val_rmse: 0.7741
Still best_val_rmse: 0.6752 (from epoch 0)

16 steps took 6.33 seconds
Epoch: 0 batch_num: 64 val_rmse: 0.6846
Still best_val_rmse: 0.6752 (from epoch 0)

16 steps took 6.33 seconds
Epoch: 0 batch_num: 80 val_rmse: 0.5838
New best_val_rmse: 0.5838

16 steps took 6.33 seconds
Epoch: 0 batch_num: 96 val_rmse: 0.507
New best_val_rmse: 0.507

16 steps took 6.33 seconds
Epoch: 0 batch_num: 112 val_rmse: 0.5091
Still best_val_rmse: 0.507 (from epoch 0)

16 steps took 6.33 seconds
Epoch: 0 batch_num: 128 val_rmse: 0.5352
Still best_val_rmse: 0.507 (from epoch 0)

16 steps took 6.54 seconds
Epoch: 1 batch_num: 3 val_rmse: 0.4937
New best_val_rmse: 0.4937

8 steps took 3.16 seconds
Epoch: 1 batch_num: 11 val_rmse: 0.5271
Still best_val_rmse: 0

Some weights of the model checkpoint at ../input/commonlitreadabilityprize/pretrained-model/clrp_roberta_base/ were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.decoder.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at ../input/commonlitreadabilityprize/pretrained-model/clrp_roberta_base/ and are newly initialized: ['roberta.pooler.dense.weight', 'rober


16 steps took 6.87 seconds
Epoch: 0 batch_num: 16 val_rmse: 0.9691
New best_val_rmse: 0.9691

16 steps took 6.33 seconds
Epoch: 0 batch_num: 32 val_rmse: 0.7235
New best_val_rmse: 0.7235

16 steps took 6.32 seconds
Epoch: 0 batch_num: 48 val_rmse: 0.7168
New best_val_rmse: 0.7168

16 steps took 6.33 seconds
Epoch: 0 batch_num: 64 val_rmse: 0.6398
New best_val_rmse: 0.6398

16 steps took 6.33 seconds
Epoch: 0 batch_num: 80 val_rmse: 0.589
New best_val_rmse: 0.589

16 steps took 6.32 seconds
Epoch: 0 batch_num: 96 val_rmse: 0.5349
New best_val_rmse: 0.5349

16 steps took 6.33 seconds
Epoch: 0 batch_num: 112 val_rmse: 0.5736
Still best_val_rmse: 0.5349 (from epoch 0)

16 steps took 6.33 seconds
Epoch: 0 batch_num: 128 val_rmse: 0.5421
Still best_val_rmse: 0.5349 (from epoch 0)

16 steps took 6.53 seconds
Epoch: 1 batch_num: 3 val_rmse: 0.5073
New best_val_rmse: 0.5073

16 steps took 6.32 seconds
Epoch: 1 batch_num: 19 val_rmse: 0.4921
New best_val_rmse: 0.4921

8 steps took 3.17 seconds


Some weights of the model checkpoint at ../input/commonlitreadabilityprize/pretrained-model/clrp_roberta_base/ were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.decoder.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at ../input/commonlitreadabilityprize/pretrained-model/clrp_roberta_base/ and are newly initialized: ['roberta.pooler.dense.weight', 'rober


16 steps took 6.86 seconds
Epoch: 0 batch_num: 16 val_rmse: 0.8881
New best_val_rmse: 0.8881

16 steps took 6.32 seconds
Epoch: 0 batch_num: 32 val_rmse: 0.6422
New best_val_rmse: 0.6422

16 steps took 6.33 seconds
Epoch: 0 batch_num: 48 val_rmse: 0.7175
Still best_val_rmse: 0.6422 (from epoch 0)

16 steps took 6.32 seconds
Epoch: 0 batch_num: 64 val_rmse: 0.6659
Still best_val_rmse: 0.6422 (from epoch 0)

16 steps took 6.32 seconds
Epoch: 0 batch_num: 80 val_rmse: 0.6194
New best_val_rmse: 0.6194

16 steps took 6.33 seconds
Epoch: 0 batch_num: 96 val_rmse: 0.5961
New best_val_rmse: 0.5961

16 steps took 6.32 seconds
Epoch: 0 batch_num: 112 val_rmse: 0.5939
New best_val_rmse: 0.5939

16 steps took 6.32 seconds
Epoch: 0 batch_num: 128 val_rmse: 0.561
New best_val_rmse: 0.561

16 steps took 6.53 seconds
Epoch: 1 batch_num: 3 val_rmse: 0.5202
New best_val_rmse: 0.5202

16 steps took 6.32 seconds
Epoch: 1 batch_num: 19 val_rmse: 0.5476
Still best_val_rmse: 0.5202 (from epoch 1)

16 steps 

# Inference

In [144]:
test_dataset = LitDataset(test_df, inference_only=True)

In [145]:
all_predictions = np.zeros((len(list_val_rmse), len(test_df)))

test_dataset = LitDataset(test_df, inference_only=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE,
                         drop_last=False, shuffle=False, num_workers=2)

for index in range(len(list_val_rmse)):            
    model_path = f"model_{index + 1}.pth"
    print(f"\nUsing {model_path}")
                        
    model = LitModel()
    model.load_state_dict(torch.load(model_path))    
    model.to(DEVICE)
    
    all_predictions[index] = predict(model, test_loader)
    
    del model
    gc.collect()


Using model_1.pth


Some weights of the model checkpoint at ../input/commonlitreadabilityprize/pretrained-model/clrp_roberta_base/ were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.decoder.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at ../input/commonlitreadabilityprize/pretrained-model/clrp_roberta_base/ and are newly initialized: ['roberta.pooler.dense.weight', 'rober


Using model_2.pth


Some weights of the model checkpoint at ../input/commonlitreadabilityprize/pretrained-model/clrp_roberta_base/ were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.decoder.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at ../input/commonlitreadabilityprize/pretrained-model/clrp_roberta_base/ and are newly initialized: ['roberta.pooler.dense.weight', 'rober


Using model_3.pth


Some weights of the model checkpoint at ../input/commonlitreadabilityprize/pretrained-model/clrp_roberta_base/ were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.decoder.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at ../input/commonlitreadabilityprize/pretrained-model/clrp_roberta_base/ and are newly initialized: ['roberta.pooler.dense.weight', 'rober


Using model_4.pth


Some weights of the model checkpoint at ../input/commonlitreadabilityprize/pretrained-model/clrp_roberta_base/ were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.decoder.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at ../input/commonlitreadabilityprize/pretrained-model/clrp_roberta_base/ and are newly initialized: ['roberta.pooler.dense.weight', 'rober


Using model_5.pth


Some weights of the model checkpoint at ../input/commonlitreadabilityprize/pretrained-model/clrp_roberta_base/ were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.decoder.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at ../input/commonlitreadabilityprize/pretrained-model/clrp_roberta_base/ and are newly initialized: ['roberta.pooler.dense.weight', 'rober

In [146]:
predictions = all_predictions.mean(axis=0)
submission_df.target = predictions
print(submission_df)
submission_df.to_csv("submission.csv", index=False)

          id    target
0  c0f722661 -0.461263
1  f0953f0a5 -0.649802
2  0df072751 -0.311437
3  04caf4e0c -2.443846
4  0e63f8bea -1.648310
5  12537fe78 -1.381306
6  965e592c0  0.144368


# Upload data

In [147]:
!date +"%Y%m%d%I%M%S"

20210704013108


In [148]:
!mkdir -p ./output/
!cp -f ./model* ./output/
!cp -f ./drive/MyDrive/kaggle/commonlit/Lightweight-Roberta-base/dataset-metadata.json ./output/
!sed -i -e "s/lightweight-roberta-base/lightweight-roberta-base-`date +"%Y%m%d%I%M%S"`/" ./output/dataset-metadata.json

In [149]:
!cat ./output/dataset-metadata.json

{
  "licenses": [
    {
      "name": "CC0-1.0"
    }
  ], 
  "id": "iamnishipy/lightweight-roberta-base-20210704013128", 
  "title": "Lightweight-Roberta-base"
}

In [150]:
!mkdir -p ./output/
!cp -f ./model* ./output/
#CHANGEME
!cp -f ./drive/MyDrive/kaggle/commonlit/Lightweight-Roberta-base/dataset-metadata-scheduler.json ./output/dataset-metadata.json
!sed -i -e "s/lightweight-roberta-base/lightweight-roberta-base-`TZ=JST-9 date +"%Y%m%d%H%M%S"`/" ./output/dataset-metadata.json
!sed -i -e "s/Lightweight-Roberta-base/Roberta-base-`TZ=JST-9 date +"%m%d%H%M%S"`/" ./output/dataset-metadata.json
!kaggle datasets create -p ./output/

Starting upload for file model_3.pth
100% 477M/477M [00:09<00:00, 50.4MB/s]
Upload successful: model_3.pth (477MB)
Starting upload for file model_5.pth
100% 477M/477M [00:11<00:00, 42.9MB/s]
Upload successful: model_5.pth (477MB)
Starting upload for file dataset-metadata-scheduler.json
100% 181/181 [00:03<00:00, 50.0B/s]
Upload successful: dataset-metadata-scheduler.json (181B)
Starting upload for file model_4.pth
100% 477M/477M [00:09<00:00, 51.7MB/s]
Upload successful: model_4.pth (477MB)
Starting upload for file model_2.pth
100% 477M/477M [00:11<00:00, 45.2MB/s]
Upload successful: model_2.pth (477MB)
Starting upload for file model_1.pth
100% 477M/477M [00:10<00:00, 46.7MB/s]
Upload successful: model_1.pth (477MB)
Your private Dataset is being created. Please check progress at /api/v1/datasets/status//iamnishipy/lightweight-roberta-base-20210704223159-scheduler


In [151]:
!cat ./output/dataset-metadata.json

{
  "licenses": [
    {
      "name": "CC0-1.0"
    }
  ], 
  "id": "iamnishipy/lightweight-roberta-base-20210704223159-scheduler", 
  "title": "Roberta-base-0704223159 with changing scheduler"
}